## 품목분류사례 chroma db 빌드

In [3]:
# !pip install jupyterlab_widgets ipywidgets

In [4]:
# pip install --upgrade pip

In [5]:
# pip install sentence_transformers

In [6]:
# pip install chromadb

In [7]:
# !pip uninstall tqdm -y
# !pip install tqdm ipywidgets
# !pip install ipywidgets

In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
import os
from tqdm.notebook import tqdm

In [9]:
df = pd.read_csv('../data/examples_cleaned.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/examples_cleaned.csv'

In [ ]:
os.makedirs("chroma_db", exist_ok=True)
# 데이터 불러오기
df = df.dropna(subset=["이름", "설명", "사유", '설명_filtered', '사유_filtered', '이름_filtered']) # NaN 제거

# 임베딩 모델 불러오기(paraphrase-multilingual-MiniLM-L12-v2)
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Chroma 클라이언트 생성 (로컬 DB)
client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection("hscode_collection")

# Batch 처리를 위한 리스트 초기화
embeddings_list = []
documents_list = []
metadatas_list = []
ids_list = []
batch_size = 500  # 성능 향상을 위한 배치 크기

print(f"--- ChromaDB 임베딩 시작: 총 {len(df)}개 항목 ---")

### 각 row 임베딩해서 저장 ###
# embedding: embedding 결과            ex. [0.1, 0.2, 0.3]
# documents: 원본 데이터               ex. "상품명: 베어링용 고무 씰 \n 설명: 철강제 링과 가황환 고무제 링이 결합된 Wheel Hub Bearing 측면 밀봉용 Seal."
# metadata: HSCode, 상품명, 시행일자   ex. {"HSCode": "1234567890", "상품명": "베어링용 고무 씰", "시행일자": "2025-01-01"}
for i, row in tqdm(df.iterrows(), total=len(df), desc="Embedding & Inserting"):
    document_text = (
        f"상품명: {row['이름']}\n"
        f"제품설명: {row['설명']}\n"
        f"분류사유: {row['사유']}\n"
        f"키워드: {row['이름_filtered']} {row['설명_filtered']} {row['사유_filtered']}"
    )
    metadata_dict = {
        "HSCode": row["HSCode"], 
        "CaseName": row["이름"],
        "Date": row["시행일자"],
        "Keywords": f"{row['설명_filtered']} {row['사유_filtered']}" # LLM에게 키워드 제공
    } # 메타데이터랑 임베딩 데이터 칼럼명 다른 거 확인!
    emb = model.encode(document_text, normalize_embeddings=True).tolist()
    
    embeddings_list.append(emb)
    documents_list.append(document_text)
    metadatas_list.append(metadata_dict)
    ids_list.append(f"case_{i}")

    # Batch 처리 로직 (Batch size에 도달하면 ChromaDB에 추가)
    if len(ids_list) >= batch_size:
        collection.add(
            embeddings=embeddings_list,
            documents=documents_list,
            metadatas=metadatas_list,
            ids=ids_list
        )
        # 리스트 초기화
        embeddings_list, documents_list, metadatas_list, ids_list = [], [], [], []

# 4. 잔여 항목 처리
if len(ids_list) > 0:
    collection.add(
        embeddings=embeddings_list,
        documents=documents_list,
        metadatas=metadatas_list,
        ids=ids_list
    )

print("✅ ChromaDB 임베딩 및 저장 완료.")
# client.persist() 는 PersistentClient를 사용하면 자동 처리됩니다.

c:\Users\user\anaconda3\envs\mw\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


--- ChromaDB 임베딩 시작: 총 28200개 항목 ---


Embedding & Inserting:   0%|          | 0/28200 [00:00<?, ?it/s]

✅ ChromaDB 임베딩 및 저장 완료.


In [ ]:
print("저장된 데이터 개수:", collection.count())

sample = collection.get(limit=3)  # 예시 데이터
print(sample)

저장된 데이터 개수: 28200
{'ids': ['case_0', 'case_1', 'case_2'], 'embeddings': None, 'documents': ['상품명: ERGOBODY SHAPY EMS BELT; EB24-SB03;\n제품설명: - 전극 패드가 내장된 직물 재질의 벨트와 컨트롤러, 충전 케이블이 지제 박스에 소매 포장되어 제시\n\n- 복부 및 허리에 착용하여, 해당 부위에 저주파 자극1)과 온열2) 제공\n* (1) 5가지 마사지 모드를 제공, EMS 기술을 이용해 근육을 반복적으로 수축·이완시켜 짧은 시간에 운동 및 마사지 효과를 제공\n(2) 3단계(38℃, 40℃, 43℃) 온열 모드 제공, 근육을 따뜻하게 풀어주어 피로 회복 및 EMS 자극의 효율을 높여주는 보조적 기능을 수행\n\n- (용도) 복부 코어 근육 강화 및 허리 마사지, 체형 관리 등 홈케어용\n\n\n< 신청물품 >\n분류사유: - 관세율표 제8543호에는 ‘그 밖의 전기기기(이 류에 따로 분류되지 않은 것으로서 고유의 기능을 가진 것으로 한정한다)’가 분류되고\n\n·같은 호 해설서에서는 “이 호에는 이 류의 다른 호에 해당되지 않고 품목분류표의 다른 류의 호에 특히 분류하지 않으며 또한 제16부나 이 류의 법정 주(Legal Note)를 적용하여도 제외하지 않는 모든 전기기기를 포함한다. … 이 호에 해당하는 전기기기는 고유의 기능을 갖지 않으면 안된다. … 이 호의 기기의 대부분의 것은 전체가 전기적으로 작동되는 전기기기나 부분품[진공관ㆍ변압기ㆍ축전기ㆍ초크(choke)ㆍ저항기 등]의 조립품으로 구성된다.”라고 설명함\n\n- 본 물품은 인체에 미세 전류를 흘려 근육을 자극시킴으로써 건강 관리에 사용되는 홈케어 제품으로, ‘이 류에 따로 분류되지 않은 고유의 기능을 가진 가정용 전기기기’에 해당하므로, 관세율표 해석에 관한 통칙 제1호와 제6호에 따라 제8543.70-2090호로 분류함\n키워드: ergobody, shapy, ems, belt, eb, sb 

### 쿼리 다르게 해서 성능 보기

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# 사용자 쿼리
query = "상품명: 베어링용 고무 씰 \n 설명: 철강제 링과 가황환 고무제 링이 결합된 Wheel Hub Bearing 측면 밀봉용 Seal."

# 쿼리 임베딩
query_emb = model.encode([query], normalize_embeddings=True).tolist()


# query = "휴대용 전기밥솥, 소형 전기 조리기기"
# query_emb = model.encode(query, normalize_embeddings=True).tolist()

# DB 불러오기
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection("hscode_collection")

# 쿼리 기반 DB 검색 결과
results = collection.query(
    query_embeddings=query_emb,
    n_results=3
)


# result 출력
for metadata, doc, dist in zip(results["metadatas"][0], results["documents"][0], results["distances"][0]):
    print("상품명:", metadata.get("CaseName"))
    print("HSCode:", metadata.get("HSCode"))
    print("시행일자:", metadata.get("Date"))
    print("키워드:", metadata.get("Keywords"))
    print("유사도 거리:", round(dist, 3))
    print("-"*50)

상품명: Rubber Seal; 300007957-0000 / F-679631.05-0061.DH.RDL;
HSCode: 8487909010
시행일자: 2025-08-14
키워드: 강제, 외관, 가황, 고무, 형상, seal, Wheel, Hub, Bearing, 측면, 밀봉, 베어링, 내부, 그리스, 외부, 누설, 외부, 물질, 침입, 차단, 수행 각종, 호로, 거나, 호로, 호로, 계류, 접속, 절연체, 코일, 접촉, 용품, 일반, 기계, 인정, 중략, 오일, oil, seal, rings, 일반, 횡단면, 원형, ring, 부분, 특징, 구조, 플렉시블, flexible, 고무, 속보, 강재, 경화, vulcanisation, 조립, 기계, 연결, 표면, 밀폐, 기름, 가스, 방지, 거나, 먼지, 방지, 가황, 연질, 고무, 금속, 결합, Wheel, Hub, Bearing, 측면, 장착, 베어링, 내부, 그리스, 외부, 누설, 외부, 물질, 침입, 차단, 목적, 오일, 보아
유사도 거리: 0.268
--------------------------------------------------
상품명: Rubber Seal; 066839483-0000 / F-573620-0061.DC.RDL;
HSCode: 8487909010
시행일자: 2025-08-18
키워드: 강제, 가황, 무제, 결합, Wheel, Hub, Bearing, 측면, 밀봉, Seal, 베어링, 내부, 그리스, 외부, 누설, 외부, 물질, 침입, 차단, 수행, 크기, 외경, mm, 두께, mm 각종, 호로, 거나, 호로, 호로, 계류, 접속, 절연체, 코일, 접촉, 용품, 전략, 일반, 기계, 인정, 특정, 기계, 외한, 조건, 비자, 동식, 윤활, 유용, 포트, 그리스, 용의, nipple, 수동식, 레버, 손잡이, 안전, 덮개, 베이스, 플레이, baseplate, 오일, oil, seal, ring, 일반, 횡단면, 원형, ring, 부분, 특징, 구조, 플렉시블, flexible, 

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# 사용자 쿼리
query = "상품명: 베어링용 고무 씰 \n 설명: 기계 베어링에 사용되는 고무재질 씰"

# 쿼리 임베딩
query_emb = model.encode([query], normalize_embeddings=True).tolist()


# query = "휴대용 전기밥솥, 소형 전기 조리기기"
# query_emb = model.encode(query, normalize_embeddings=True).tolist()

# DB 불러오기
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection("hscode_collection")

# 쿼리 기반 DB 검색 결과
results = collection.query(
    query_embeddings=query_emb,
    n_results=3
)


# result 출력
for metadata, doc, dist in zip(results["metadatas"][0], results["documents"][0], results["distances"][0]):
    print("상품명:", metadata.get("CaseName"))
    print("HSCode:", metadata.get("HSCode"))
    print("시행일자:", metadata.get("Date"))
    print("키워드:", metadata.get("Keywords"))
    print("유사도 거리:", round(dist, 3))
    print("-"*50)

상품명: 똘똘이시럽쭉쭉아이스크림가게
HSCode: 9503003919
시행일자: 2019-01-21
키워드: 계산, 모형, 완구, 멜로디, 버튼, 가지, 멘트, 노래, OPEN, 스위치, 현금, 수납, 놀이, 용품 가황, 배합, 고무, 일차, primary, form, 시트, sheet, 스트립, 소호, 카본블랙, 실리카, 배합, 가황, 배합, 고무, 로서, 일차, primary, form, 이나, plate, 시트, sheet, 스트립, strip, 카본블랙, carbon, black, 이나, 실리카, silica, 배합, 고무, 광유, mineral, oil, 성분, 배합, 인지, 가목, 응고, 전후, 가황제, 가황, 촉진, 지연제, 성제, 프리, 이즈, pre, vulcanised, 고무, 라텍스, 첨가, 외한, 안료, 착색제, 식별, 첨가, 외한, 가소제, 증량제, 유전, 고무, 광유, mineral, oil, 외한, 충전, 강제, 유기, 용제, 물질, 나목, 물질, 외한, 배합, 고무, 고무, 혼합물, 적용, 참고, 가황, 가황제, 보통, 특정, 물질, 가황, 촉진, 성제, 지연제, 가소제, plasticizer, 증량제, 충전, 보강, 제나, 나목, 첨가제, 첨가, 가황, 혼합물, 배합, 고무, compounded, rubber, 간주, 합성, 고무, 카본블랙, 실리카, 혼합, 상의, 가황, 배합, 고무
유사도 거리: 0.337
--------------------------------------------------
상품명: BOLT
HSCode: 7318152000
시행일자: 2016-11-25
키워드: 육각형, 두부, 나선, 가공, 끝단, 모양, 강제, 볼트, 차량, 부품, 체결, 사진, 볼트, 제외 나목, 비금속, 범용, 이나, 플라스틱, 제외, 가목, 비금속, 범용, 철강, 스크루, screw, 볼트, bolt, 너트, nut, 코치, 스크루, coach, screw, 스크루, screw, hook, 리벳, rivet, 코터, cotter,

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# 사용자 쿼리
query = "베어링 기계용 고무 씰링 재질"

# 쿼리 임베딩
query_emb = model.encode([query], normalize_embeddings=True).tolist()


# query = "휴대용 전기밥솥, 소형 전기 조리기기"
# query_emb = model.encode(query, normalize_embeddings=True).tolist()

# DB 불러오기
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection("hscode_collection")

# 쿼리 기반 DB 검색 결과
results = collection.query(
    query_embeddings=query_emb,
    n_results=3
)


# result 출력
for metadata, doc, dist in zip(results["metadatas"][0], results["documents"][0], results["distances"][0]):
    print("상품명:", metadata.get("CaseName"))
    print("HSCode:", metadata.get("HSCode"))
    print("시행일자:", metadata.get("Date"))
    print("키워드:", metadata.get("Keywords"))
    print("유사도 거리:", round(dist, 3))
    print("-"*50)

상품명: 완구 ; 차영웅 스타터 세트
HSCode: 9503003700
시행일자: 2021-09-23
키워드: 별도, 소매, 포장, 카드, 바이트, 울프, 세트, 터닝카, 스타, 버닝, 세트, 제품, 세트, 재포, 연령, 이상, 카드, 바이트, 울프, 바이트, 울프, 카드, 카드, 스토퍼, 마법진, 카드, 놀이, 설명, 서로, 구성, 터닝카, 스타, 버닝, 터닝카, 스타, 카드, 클립, 스티커, 구성, 놀이, 방법, 놀이, 터닝카, 바이어, 울프, 슈팅, 가운데, 위치, 카드, 먼저, 캐치, 사람, 카드, 카드, 상대, 먼저, 사람, 승리, 터닝카, 추가 자전거, 스쿠터, 페달, 자동차, 바퀴, 완구, 인형, 인형, 완구, 축소, 모형, 오락, 모형, 작동, 인지, 각종, 퍼즐, 본질, 사람, 어린이, 어른, 오락, 위해, 의도, 완구, 완구, 무기, 공구, 정원, 세트, 장난감, 병정, 세트, sets, 애니메이션, 카드, 게임, 위해, 세트, 아웃, 피트, 완구
유사도 거리: 0.44
--------------------------------------------------
상품명: 아머피트 레드; 중국
HSCode: 9503003919
시행일자: 2017-01-17
키워드: 피트, 레드, 자동차, 장갑차, 카드, 소매, 세트, 포장, 애니메이션, 캐릭터, 어린이, 완구, 재현, 자동차, 장갑차, 내장, 하단, 철심, 부분, 카드, 철심, 형태, lock, 해제, 자동차, 상단, 내장, 장갑차, 공중, 발사, 형태, 완구, 연령, 이상 자전거, 스쿠터, 페달, 자동차, 바퀴, 완구, 인형, 용의, 인형, 기타, 완구, 축소, 모형, 오락, 모형, 작동, 인지, 여부, 불문, 각종, 퍼즐, 완구, 대하, 본질, 사람, 어린이, 어른, 오락, 위해, 의도, 완구, 완구, 차량, 그룹, 이외, 기차, 전기, 여부, 불문, 비행기, 보트, 부속품, 철로, 호기, 예시, 자동차, 카드, 이용, 장갑차, 발사, 흥미, 유발, 의도, 기타, 완구, 보아, 용어
유사도 

In [ ]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import chromadb
from tqdm import tqdm

FILE_PATH = "D:/2025-2-DSCD-FIVE-01/data/HS_code_Nomenclature.md"

# 1) 파일 읽기 및 페이지 분할
with open(FILE_PATH, "r", encoding="utf-8") as f:
    text = f.read()

SEPARATOR = "______________"
pages_raw = text.split(SEPARATOR)
pages = [p.strip() for p in pages_raw if p.strip()]

print(f"총 페이지 수: {len(pages)}")

# 2) 텍스트 분할기 설정 (chunk size 및 overlap 조절 가능)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    length_function=len
)

# 3) OpenAI 임베딩 모델 준비
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# 4) ChromaDB PersistentClient 및 컬렉션 생성
client = chromadb.PersistentClient(path="D:/2025-2-DSCD-FIVE-01/data/nomenclature_chroma_db", normalize_embeddings=True)
collection = client.get_or_create_collection("hscode_nomenclature")

# 5) batch 저장용 리스트 초기화
embeddings_list = []
documents_list = []
metadatas_list = []
ids_list = []
batch_size = 200

print(f"--- ChromaDB 임베딩 시작 (총 {len(pages)}페이지, 내부 chunking 적용) ---")

# 6) 페이지별로 chunking + 임베딩 + 저장
for i, page in tqdm(enumerate(pages), desc="Processing Pages"):

    # 페이지 내 텍스트를 chunk 단위로 분할
    chunks = splitter.split_text(page)
    
    for j, chunk in enumerate(chunks):
        document_text = chunk
        metadata_dict = {
            "page": i + 1,            # 원본 페이지 번호
            "chunk": j + 1,           # 페이지 내 chunk 번호
            "source_file": FILE_PATH
        }

        # 임베딩 생성 (OpenAIEmbeddings는 리스트 입력)
        emb = embedding.embed_documents([document_text])[0]

        embeddings_list.append(emb)
        documents_list.append(document_text)
        metadatas_list.append(metadata_dict)
        ids_list.append(f"page_{i+1}_chunk_{j+1}")

        # batch 단위 저장
        if len(ids_list) >= batch_size:
            collection.add(
                embeddings=embeddings_list,
                documents=documents_list,
                metadatas=metadatas_list,
                ids=ids_list
            )
            embeddings_list, documents_list, metadatas_list, ids_list = [], [], [], []

# 7) 마지막 batch 저장
if len(ids_list) > 0:
    collection.add(
        embeddings=embeddings_list,
        documents=documents_list,
        metadatas=metadatas_list,
        ids=ids_list
    )

print("✅ Markdown 페이지 + chunk 단위 ChromaDB 구축 완료!")


c:\Users\user\anaconda3\envs\mw\lib\site-packages\langchain_openai\embeddings\base.py:306: UserWarning: WARNING! normalize_embeddings is not default parameter.
                    normalize_embeddings was transferred to model_kwargs.
                    Please confirm that normalize_embeddings is what you intended.
  warnings.warn(


총 페이지 수: 203
--- ChromaDB 임베딩 시작 (총 203페이지, 내부 chunking 적용) ---


Processing Pages: 0it [00:00, ?it/s]

Processing Pages: 0it [00:00, ?it/s]


TypeError: Embeddings.create() got an unexpected keyword argument 'normalize_embeddings'